In [1]:
# Dependencies
import pandas as pd
import requests
from pprint import pprint
import pymongo
from bs4 import BeautifulSoup as bs
from splinter import Browser
from flask import Flask, render_template
import lxml
import time

# Nasa's Mars News Site
url = "https://mars.nasa.gov/news/"

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
# Set up MongoDB
conn='mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database
db = client.missions_to_mars
collection = db.mars_news

In [4]:
# Retrieve page with the requests module
browser.visit(url)
time.sleep(2)
html = browser.html
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(html, 'html.parser')


In [5]:
# Assign variables

item_list=soup.find('ul', class_='item_list').find('li', class_='slide')

news_title = item_list.find('div', class_='content_title').get_text()
news_p = item_list.find('div', class_='article_teaser_body').get_text()
print(news_title)
print(news_p)

The Man Who Wanted to Fly on Mars
The Mars Helicopter is riding to the Red Planet this summer with NASA's Perseverance rover. The helicopter's chief engineer, Bob Balaram, shares the saga of how it came into being.


In [6]:
# Add to MongoDB
post = {
    'title': news_title,
    'paragraph': news_p
}

collection.insert_one(post)

In [7]:
# Splinter to featured image
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_image_url)
element = browser.find_by_id('full_image')
element.click()
browser.is_element_present_by_text('more info', wait_time = 1)
more_info_element = browser.links.find_by_partial_text('more info')
more_info_element.click()

html = browser.html
img = bs(html, 'html.parser')
img_url = img.select_one('figure.lede a img').get('src')
img_url

'/spaceimages/images/largesize/PIA17448_hires.jpg'

In [8]:
# Twitter - code from Vicky and Tracy, after a lot of troubleshootong!
all_tweets = []
twitter_url = 'https://twitter.com/marswxreport?lang=en'
data = requests.get(twitter_url)
html = bs(data.text, 'html.parser')
timeline = html.select('#timeline li.stream-item')
for tweet in timeline:
    tweet_id = tweet['data-item-id']
    tweet_text = tweet.select('p.tweet-text')[0].get_text()
    all_tweets.append({"id": tweet_id, "text": tweet_text})
    print(all_tweets)
    break
    
# Add to MongoDB
post = {
    'tweet': tweet_text
}

collection.insert_one(post)

[{'id': '1245332913786253312', 'text': 'InSight sol 478 (2020-03-31) low -93.3ºC (-135.9ºF) high -8.5ºC (16.7ºF)\nwinds from the S at 6.1 m/s (13.6 mph) gusting to 18.1 m/s (40.5 mph)\npressure at 6.50 hPapic.twitter.com/hlo9Yr5bKo'}]


In [9]:
# Mars Facts table
facts_url = 'https://space-facts.com/mars/'
mars_table = pd.read_html(facts_url)[0]
mars_table.columns = ['description', 'value']
mars_table

,description,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [10]:
mars_table.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>description</th>\n      <th>value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium 

In [11]:
#Hemisphere images
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)
links = browser.find_by_css('a.product-item h3')

hemispheres = []

for url_id in range(len(links)):
    hemisphere = {}
    browser.find_by_css('a.product-item h3')[url_id].click()
    sample = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample['href']
    hemisphere['title'] = browser.find_by_css('h2.title').text
    hemispheres.append(hemisphere)
    browser.back()
    
hemispheres

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [12]:
browser.quit()